**General and metadata-field search, &posting metadata examples:**

In [2]:
import requests
import json
import os
earthcube_host=os.getenv('earthcube_host')
my_host=os.getenv('my_host')



**Below is where you enter your free-text search term:**

In [3]:
qry_str = "multibeam sonar" #free txt search term.   #urlencoded in requests

I have a search [layer](my_host + "/ext/ec/search/") that allows for: earthcube_host + "/search/carbon" w/o the need for an auth-key, &allows for [do](https://github.com/MBcode/ec/blob/master/search/sc.py)ing things like below right away

In [4]:
ecr = requests.get(f"{earthcube_host}/search/{qry_str}")
print(json.dumps(ecr.json(), indent=2))

{
  "count": 11,
  "size": 11,
  "scanned_size": 240,
  "results": [
    {
      "id": "5f5fcf375e0e04ebc01c4050",
      "name": " Processed Multibeam Sonar Data (version 2) near the Antarctic Peninsula acquired during Nathaniel B. Palmer expedition NBP0107 (2001)",
      "description": "Data DOI: 10.1594/IEDA/100307 \nThis data set was acquired with a SeaBeam Instruments model 2112 multibeam sonar system during Nathaniel B. Palmer expedition NBP0107 conducted in 2001 (Chief Scientist: Dr. Eugene Domack). These data files include Acoustic Backscatter, Swath Bathymetry data and were processed at sea using the open source MB-System multibeam sonar processing software. These data are included in the GMRT Synthesis, beginning with version 1. Data were acquired as part of the project(s): Global Climate Change and the Evolutionary Ecology of Antarctic Mollusks in the Late Eocene; Development of a Luminescence Dating Capability for Antarctic Glaciomarine Sediments: Tests of Signal Zeroing at 

=Everything below this can be put in above layer, including metadata search via the elasticsearch [syntax-guide](https://github.com/clowder-framework/clowder/blob/develop/app/views/searchResults.scala.html#L35).

Next I will copy over the discovery work from the other [NoteBook](https://colab.research.google.com/drive/14_bTSVMwLk-XbmIeIgAFnswv250oRGOK#scrollTo=-Ar91tVRTuMu) &can easily make into another search-api option [here](https://github.com/MBcode/ec/blob/master/search/ss.py)

In [ ]:
#This code runs off the triplestore search results, 
#but can run off this backend as well w/some small mods
#---
#for result in r.json()["results"]:
# snippit=result['id']+" "+result['description']
# url=result['url']. #set this key but might get elsewhere
#now for output to be clustered by 'brew carrot2' service

cout = """<?xml version="1.0" encoding="UTF-8"?>
<searchresult>
  <query>"""
print(cout,qry_str,"</query>")
i=0
for result in results["results"]["bindings"]:
    print("<document id=\"",i,"\">")
    print("<url>",result["s"]["value"],"</url>")
    print("<snippet>",result["o"]["value"],"</snippet></document>")
    i=i+1
print("</searchresult>")

Output of running carrot2(dcs)service,on xml output: my_host + "/ext/ec/carbon2.json"

Want [search.carrot2.org/#/search/web/carbon/treemap](https://search.carrot2.org/#/search/web/site%3Ahttps%3A%2F%2Fwww.hydroshare.org%20carbon/treemap) viz after sending xml above to clustering service.

Now that there is some redirection [code](https://github.com/MBcode/ec/tree/master/search) you will be able to get it directly

In [ ]:
#ecr = requests.get(f"{earthcube_host}/search3/{qry_str}")


In [ ]:
#clowder_host = "https://clowder.ncsa.illinois.edu/clowder"
clowder_host = "https://earthcube.clowderframework.org"

In [ ]:
import os
clowder_key = os.getenv('testkey') #get this to work before sharing, well will be a diff key for others
clowder_key = "xxx" #since we will get an 'open' instance will we still need?

In [ ]:
clowder_key = os.getenv('eckey') #I can use locally w/new instance till it is fixed
#r = requests.get(f"{clowder_host}/api/search?query={qry_str}") #looks like still need a login, Need clowder fix4this

In [ ]:
#print(json.dumps(r.json(), indent=2))

In [ ]:
#r = requests.get(f"{clowder_host}/api/search?query={qry_str}&key={clowder_key}") 
r = requests.get(f"{clowder_host}/api/search?query={qry_str}", headers={'X-API-Key': clowder_key})

This free-text search call is not that dissimilar from the present geodex api.

In [ ]:
print(json.dumps(r.json(), indent=2))

{
  "count": 3,
  "size": 3,
  "scanned_size": 240,
  "results": [
    {
      "id": "5f5fcf375e0e04ebc01c4050",
      "name": " Processed Multibeam Sonar Data (version 2) near the Antarctic Peninsula acquired during Nathaniel B. Palmer expedition NBP0107 (2001)",
      "description": "Data DOI: 10.1594/IEDA/100307 \nThis data set was acquired with a SeaBeam Instruments model 2112 multibeam sonar system during Nathaniel B. Palmer expedition NBP0107 conducted in 2001 (Chief Scientist: Dr. Eugene Domack). These data files include Acoustic Backscatter, Swath Bathymetry data and were processed at sea using the open source MB-System multibeam sonar processing software. These data are included in the GMRT Synthesis, beginning with version 1. Data were acquired as part of the project(s): Global Climate Change and the Evolutionary Ecology of Antarctic Mollusks in the Late Eocene; Development of a Luminescence Dating Capability for Antarctic Glaciomarine Sediments: Tests of Signal Zeroing at th

The search results from API need the *detials* [added](https://github.com/MBcode/ec/blob/master/search/sc.py) to them:

In [ ]:
#getLD the 'details' for each search result, and put link back in search results
#for result in r.json()["results"]:
#  dataset=result['id']
#  print(dataset)
#  LD=getLD(dataset) #mv fnc up or eval 1st
# #could turn into jsonl-ld playground viz tab url here
#  result['details']=LD

Here is a [picture](http://isda.ncsa.illinois.edu/~mbobak/png/cms.png) of the ***(metadata) search interface*** after you log in.  It shows searching over the different *metadata facets*; *Which can be had via the key-value search below*: 

**Can also give key-value pairs to search**,  &post the LD as well.

In [ ]:
metadata_qry = "\"Audience\":\"geoscience\""
r2 = requests.get(f"{clowder_host}/api/search?query={metadata_qry}", headers={'X-API-Key': clowder_key})
print(json.dumps(r2.json(), indent=2))

{
  "count": 1,
  "size": 1,
  "scanned_size": 240,
  "results": [
    {
      "id": "5f4eb5045e0efb9145fc31b0",
      "name": " Processed Multibeam Sonar Data (version 1) near the Antarctic Peninsula acquired during Nathaniel B. Palmer expedition NBP0104 (2001)",
      "description": "\nThis data set was acquired with a SeaBeam Instruments model 2112 multibeam sonar system during Nathaniel B. Palmer expedition NBP0104 conducted in 2001 (Chief Scientist: Dr. Peter Wiebe). These data files include Acoustic Backscatter, Swath Bathymetry data and were processed at sea using the open source MB-System multibeam sonar processing software. These data are included in the GMRT Synthesis, beginning with version 1. Data were acquired as part of the project(s): SO GLOBEC: Water-column nutrient analysis; SO GLOBEC: Water-column phytoplankton primary productivity; SO GLOBEC: Seabird distribution and abundance; SO GLOBEC: Krill distribution, physiology, and predations; Copepod prey abundance; SO GLOB

**Able to POST jsonLD, so will be ready for when we get *open* radiant instance this week**:

In [ ]:
from google.colab import files
uploaded = files.upload() #condidering using, so you could try this w/any local jsonLD file

In [ ]:
#make sure the input fn's have been milled w/something like:
# alias dcat2i "sed -i '/http:..www.w3.org.ns.dcat#/s//dcat:/g'"

#This does everything below in one go
# except getLD(datasetID) which you might also use from search results
def jsonLD2dataset(fn):
  "get name&desc(=title&abstract)make clowder dataset &assert associated jsonLD"
  with open(fn) as f:
    jsLD=f.read()
  d=json.loads(jsLD)  #will have2do the sed like fixes, &get that in the 2nd post,2fix exceptions
  #metadata2={'name': d['name'], 'description': d['description']} #url next
  #metadata3={'name': d['name'], 'description': d['description'], 'url': d['identifier'][0]['url']}
  clowder_space = "5f6d2340e4b010d24f82045b"
  metadata3={'name': d['name'], 'description': d['description'], 'url': d['identifier'][0]['url'],'space': [clowder_space], 'access': 'PUBLIC'}
  print(metadata3)
  headers3={'X-API-Key': clowder_key,'accept': '*/*', 'Content-Type': 'application/json'}
  rds=requests.post(f'{clowder_host}/api/datasets/createempty',headers=headers3, data=json.dumps(metadata3))
  print(rds.json())
  datasetID=rds.json()['id']
  print(datasetID)
  headers2={'X-API-Key': clowder_key, 'Content-Type': 'application/json'}
  url=f'{clowder_host}/api/datasets/{datasetID}/metadata'
  print(url)
  r=requests.post(url,data=jsLD,headers=headers2)
  print(r.json())
  return datasetID



>*Create dataset stub for the repo's landing page DOI:*

>curl -X POST "https://clowder.ncsa.illinois.edu/clowder/api/datasets/createempty" -H "accept: \*\/\*" -H "X-API-Key: $clowder_key" -H "Content-Type: application/json" -d "{\"name\":\"http://dx.doi.org/10.1594/IEDA/300002\"}"

>{ "id": "5f6108b15e0e04ebc01d6436" }

In [ ]:
def createDS(fd):
  "dict of file metadata to init an clowder dataset"
  values = json.dumps(fd)
  r=requests.post(f'{clowder_host}/api/datasets/createempty',headers={'X-API-Key': clowder_key,'accept': '*/*', 'Content-Type': 'application/json'}, data=values)
  print(r.json())

def createDS2(name,description):  #not needed
  "enter DOI and abstract to get the dataset page going"
  fd = {"name": name, "description": description}
  createDS(fd)

def ld2dct(fn):
  "get title&abstract from jsonLD into a dict"
  with open(fn) as f:
    d=json.loads(f.read())
    r={'name': d['name'], 'description': d['description']} #want url too
    #r={'name': d['name'], 'description': d['description'], 'url': d['identifier'][0]['url']}
    #print(r)
    return r
#ld2dct('ld/310068.jsonld')
#{'name': 'Processed ship-based ADCP Sonar Data from the East Pacific Rise at 9N acquired during the Atlantis expedition AT15-32 (2008)', 'description': 'Abstract: This data set was acquired with a ship-based RDI OS-75 ADCP Sonar during Atlantis expedition AT15-32 conducted in 2008 (Chief Scientist: Dr. George Luther). These data files are of Binary format and include Current Measurement data and were processed after data collection. Data were acquired as part of the project(s): IDEA: Coupling Communication Technology with in Situ Voltammetric Analyzers for Real Time and Unattended Monitoring of Biogeochemically Relevant Chemical Species and Establishing a Long-Term Geodetic Network at the East Pacific Rise Ridge 2000 Integrated Studies Site, and funding was provided by NSF grant(s): OCE03-08398 and OCE06-47840.'}

def ld2ds(fn):
  "get title&abstract from jsonLD file to create a dataset"
  createDS(ld2dct(fn))


In [ ]:
createDS2("http://dx.doi.org/10.1594/IEDA/300002", "abstract-here")

{'id': '5f6a70385e0e69663bea65bd'}





>*Add it's jsonLD to the dataset_id:*

>curl -X POST -H "Content-Type: application/json" -d @300002.jsonld "https://clowder.ncsa.illinois.edu/clowder/api/datasets/5f60ce0d5e0e04ebc01d2dae/usermetadata?key=$clowder_key"

>{"status":"success"}

> *Coming here in [code](https://github.com/MBcode/ec) soon, but still will run from server w/the jsonLD cache.*

>**Use**: jsonLD2dataset above, and: getLD below

In [ ]:
#add function to post the jsonLD as above
def postLD(datasetID,fn):  
    files = {'jsonLD':(fn,open(fn,'rb'))}
    r=requests.post(f'{clowder_host}/api/datasets/{datasetID}/usermetadata',files=files,headers={'X-API-Key': clowder_key, 'Content-Type': 'application/json'})
    print(r.json)
  #soon locally: postLD('5f6a70385e0e69663bea65bd', 'ld/300002.jsonld')

In [ ]:
#Then to check it was uploaded, or when get search results, &want metadata, send an 'id' to this:
def getLD(datasetID):  
    r = requests.get(f'{clowder_host}/api/datasets/{datasetID}/metadata', headers={'X-API-Key' : clowder_key})
    print(json.dumps(r.json(), indent=2))

In [ ]:
getLD("5f60ce0d5e0e04ebc01d2dae")

[
  {
    "id": "5f4eb3935e0efb9145fc3069",
    "spaceId": "5f4eb3935e0efb9145fc3064",
    "json": {
      "label": "Abstract",
      "uri": "http://purl.org/dc/terms/abstract",
      "type": "string"
    }
  },
  {
    "id": "5f4eb3935e0efb9145fc306a",
    "spaceId": "5f4eb3935e0efb9145fc3064",
    "json": {
      "label": "Alternative Title",
      "uri": "http://purl.org/dc/terms/alternative",
      "type": "string"
    }
  },
  {
    "id": "5f4eb3935e0efb9145fc306b",
    "spaceId": "5f4eb3935e0efb9145fc3064",
    "json": {
      "label": "Audience",
      "uri": "http://purl.org/dc/terms/audience",
      "type": "string"
    }
  },
  {
    "id": "5f4eb3935e0efb9145fc306c",
    "spaceId": "5f4eb3935e0efb9145fc3064",
    "json": {
      "label": "CSDMS Standard Name",
      "uri": "http://csdms.colorado.edu/wiki/CSN_Searchable_List",
      "type": "list",
      "definitions_url": "http://ecgs.ncsa.illinois.edu/gsis/CSN"
    }
  },
  {
    "id": "5f4eb3935e0efb9145fc306d",
    "spaceI

**Previously**: You should also have access to the old/original search [notebook](https://colab.research.google.com/drive/14_bTSVMwLk-XbmIeIgAFnswv250oRGOK#scrollTo=cmxAMz308j-s) that did free-text, allowed for faceted metadata, & has a **clustered** (used for *discovery*) search results example..  It uses a query to blazegraph, just like geodex was moving towards.  Also for 2ndry use, want to do some sci-on-schema.org variableMeasured work like this EC-[NoteBook](https://github.com/clowder-framework/ec20_marini_etal/blob/master/ecgs.ipynb) submission.